# ResNet
当大家还在惊叹 GoogLeNet 的 inception 结构的时候，微软亚洲研究院的研究员已经在设计更深但结构更加简单的网络 ResNet，并且凭借这个网络子在 2015 年 ImageNet 比赛上大获全胜。

ResNet 有效地解决了深度神经网络难以训练的问题，可以训练高达 1000 层的卷积网络。网络之所以难以训练，是因为存在着梯度消失的问题，离 loss 函数越远的层，在反向传播的时候，梯度越小，就越难以更新，随着层数的增加，这个现象越严重。之前有两种常见的方案来解决这个问题：

1.按层训练，先训练比较浅的层，然后在不断增加层数，但是这种方法效果不是特别好，而且比较麻烦

2.使用更宽的层，或者增加输出通道，而不加深网络的层数，这种结构往往得到的效果又不好

ResNet 通过引入了跨层链接解决了梯度回传消失的问题。

![](https://ws1.sinaimg.cn/large/006tNc79ly1fmptq2snv9j30j808t74a.jpg)

这就普通的网络连接跟跨层残差连接的对比图，使用普通的连接，上层的梯度必须要一层一层传回来，而是用残差连接，相当于中间有了一条更短的路，梯度能够从这条更短的路传回来，避免了梯度过小的情况。

假设某层的输入是 x，期望输出是 H(x)， 如果我们直接把输入 x 传到输出作为初始结果，这就是一个更浅层的网络，更容易训练，而这个网络没有学会的部分，我们可以使用更深的网络 F(x) 去训练它，使得训练更加容易，最后希望拟合的结果就是 F(x) = H(x) - x，这就是一个残差的结构

残差网络的结构就是上面这种残差块的堆叠，下面让我们来实现一个 residual block

In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf
from utils import cifar10_input

In [ ]:
batch_size = 64

train_imgs, train_labels, val_imgs, val_labels = cifar10_input.load_data(data_dir='cifar10_data', image_size=96)

In [ ]:
import tensorflow.contrib.slim as slim

### 首先定义一个下采样函数

In [ ]:
def subsample(x, factor, scope=None):
    if factor == 1:
        return x
    return slim.max_pool2d(x, [1, 1], factor, scope=scope)

### 定义一个`residual_block`

In [ ]:
def residual_block(x, bottleneck_depth, out_depth, stride=1, scope='residual_block'):
    in_depth = x.get_shape().as_list()[-1]
    with tf.variable_scope(scope):
        # 如果通道数没有改变,用下采样改变输入的大小
        if in_depth == out_depth:
            shortcut = subsample(x, stride, 'shortcut')
        # 如果有变化, 用卷积改变输入的通道以及大小
        else:
            shortcut = slim.conv2d(x, out_depth, [1, 1], stride=stride, activation_fn=None, scope='shortcut')

        residual = slim.conv2d(x, bottleneck_depth, [1, 1], stride=1, scope='conv1')
        residual = slim.conv2d(residual, bottleneck_depth, 3, stride, scope='conv2')
        residual = slim.conv2d(residual, out_depth, [1, 1], stride=1, activation_fn=None, scope='conv3')

        # 相加操作
        output = tf.nn.relu(shortcut + residual)

        return output

### 构建`resnet`整体结构

In [ ]:
def resnet(inputs, num_classes, reuse=None, is_training=None, verbose=False):
    with tf.variable_scope('resnet', reuse=reuse):
        net = inputs
        
        if verbose:
            print('input: {}'.format(net.shape))
        
        with slim.arg_scope([slim.batch_norm], is_training=is_training):
            with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], padding='SAME'):
                with tf.variable_scope('block1'):
                    net = slim.conv2d(net, 32, [5, 5], stride=2, scope='conv_5x5')

                    if verbose:
                        print('block1: {}'.format(net.shape))
                    
                with tf.variable_scope('block2'):
                    net = slim.max_pool2d(net, [3, 3], 2, scope='max_pool')
                    net = residual_block(net, 32, 128, scope='residual_block1')
                    net = residual_block(net, 32, 128, scope='residual_block2')

                    if verbose:
                        print('block2: {}'.format(net.shape))
                    
                with tf.variable_scope('block3'):
                    net = residual_block(net, 64, 256, stride=2, scope='residual_block1')
                    net = residual_block(net, 64, 256, scope='residual_block2')

                    if verbose:
                        print('block3: {}'.format(net.shape))
                    
                with tf.variable_scope('block4'):
                    net = residual_block(net, 128, 512, stride=2, scope='residual_block1')
                    net = residual_block(net, 128, 512, scope='residual_block2')

                    if verbose:
                        print('block4: {}'.format(net.shape))
                
                with tf.variable_scope('classification'):
                    net = tf.reduce_mean(net, [1, 2], name='global_pool', keep_dims=True)
                    net = slim.flatten(net, scope='flatten')
                    net = slim.fully_connected(net, num_classes, activation_fn=None, normalizer_fn=None, scope='logit')

                    if verbose:
                        print('classification: {}'.format(net.shape))
                    
                return net

In [ ]:
with slim.arg_scope([slim.conv2d], activation_fn=tf.nn.relu, normalizer_fn=slim.batch_norm) as sc:
    conv_scope = sc

In [ ]:
is_training = tf.placeholder(tf.bool, name='is_training')

with slim.arg_scope(conv_scope):
    train_out = resnet(train_imgs, 10, is_training=is_training, verbose=True)
    val_out = resnet(val_imgs, 10, is_training=is_training, reuse=True)

### 构建训练

In [ ]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

In [ ]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('val'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

In [ ]:
lr = 0.01

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)

In [ ]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss)

### 开始训练

In [ ]:
from utils.learning import train_with_bn

In [ ]:
train_with_bn(train_op, train_loss, train_acc, val_loss, val_acc, 20000, is_training)

### 小练习
- 写一个自己的训练函数, 然后用`tensorboard`打开模型文件，看看每个residual_block底下卷积层各个部分的位置，是否是conv->bn->relu？
- 尝试修改卷积层中各个部分的顺序为bn->relu->conv